In [28]:
import os
from PIL import Image, ImageFont, ImageDraw

if not os.path.exists('C:/Users/MANUEL/Desktop/handwritten/habbakuk/'):
        os.makedirs('C:/Users/MANUEL/Desktop/handwritten/habbakuk/')


path_pretrain_habbakuk="C:/Users/MANUEL/Desktop/handwritten/habbakuk"

#Load the font and set the font size to 42
font = ImageFont.truetype('C:/Users/MANUEL/Desktop/handwritten/Habbakuk.ttf', 42)

#Character mapping for each of the 27 tokens
char_map = {'Alef' : ')', 
            'Ayin' : '(', 
            'Bet' : 'b', 
            'Dalet' : 'd', 
            'Gimel' : 'g', 
            'He' : 'x', 
            'Het' : 'h', 
            'Kaf' : 'k', 
            'Kaf-final' : '\\', 
            'Lamed' : 'l', 
            'Mem' : '{', 
            'Mem-medial' : 'm', 
            'Nun-final' : '}', 
            'Nun-medial' : 'n', 
            'Pe' : 'p', 
            'Pe-final' : 'v', 
            'Qof' : 'q', 
            'Resh' : 'r', 
            'Samekh' : 's', 
            'Shin' : '$', 
            'Taw' : 't', 
            'Tet' : '+', 
            'Tsadi-final' : 'j', 
            'Tsadi-medial' : 'c', 
            'Waw' : 'w', 
            'Yod' : 'y', 
            'Zayin' : 'z'}

#Returns a grayscale image based on specified label of img_size
def create_image(label, img_size):
    if (label not in char_map):
        raise KeyError('Unknown label!')

    #Create blank image and create a draw interface
    img = Image.new('L', img_size, 255)    
    draw = ImageDraw.Draw(img)

    #Get size of the font and draw the token in the center of the blank image
    w,h = font.getsize(char_map[label])
    draw.text(((img_size[0]-w)/2, (img_size[1]-h)/2), char_map[label], 0, font)

    return img

#Creates a 50x50 images of all the hebrew alphabet and save it to disk
#To get the raw data cast it to a numpy array

for character in list(char_map.keys()):
    img = create_image(character, (50, 50))
    os.mkdir(os.path.join(path_pretrain_habbakuk, character))
    img.save(f'C:/Users/MANUEL/Desktop/handwritten/habbakuk/{character}/example_{character}.png')


In [39]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

In [40]:
batch_size=12
input_size = (64, 64)
optimizer = 'adam'

In [41]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  1


RuntimeError: Physical devices cannot be modified after being initialized

In [42]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy

In [43]:
import keras

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
import numpy as np
image_shape = (64,64,3)
np.random.seed(1000)
model = Sequential()
#First Convolutional layer
model.add(Conv2D(filters=96, input_shape=image_shape, kernel_size=(11,11), strides=(4,4),padding='valid'))
model.add(Activation('relu'))

#Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

#Second Convolutional layer
model.add(Conv2D(filters=96, kernel_size=(5,5), strides=(1,1), padding='same'))
model.add(Activation('relu'))

#Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))

#Third Convolutional layer
model.add(Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

#Fourth Convolutional layer
model.add(Conv2D(filters=16, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

#Fifth Convolutional layer
model.add(Conv2D(filters=22, kernel_size=(3,3), strides=(1,1), padding='same'))
model.add(Activation('relu'))

#Max Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same'))

#Passing it to a fully connected layer, here we do the flatten!
model.add(Flatten())

#First Fully Connected layer has 4096 neurons
model.add(Dense(300, input_shape=(64*64*3,)))
model.add(Activation('relu'))

#Add dropout to prevent overfitting
model.add(Dropout(0.5))

#Second Fully Connected layer
model.add(Dense(200))
model.add(Activation('relu'))

#Add Dropout
model.add(Dropout(0.5))

#Output layer
model.add(Dense(27))
model.add(Activation('softmax'))

model.summary()

model.compile(loss=keras.losses.categorical_crossentropy, optimizer='adam', metrics=["accuracy"])



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 14, 14, 96)        34944     
_________________________________________________________________
activation_8 (Activation)    (None, 14, 14, 96)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 96)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 96)          230496    
_________________________________________________________________
activation_9 (Activation)    (None, 6, 6, 96)          0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 2, 2, 96)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 2, 2, 16)         

In [61]:
gen_train = ImageDataGenerator(rescale=1./255, width_shift_range=0.1,
        shear_range=0.15, zoom_range=0.1,
        channel_shift_range=10., horizontal_flip=False, validation_split=0.2)

In [62]:
gen_test = ImageDataGenerator(rescale=1./255)

In [63]:
train_generator = gen_train.flow_from_directory(
                path_pretrain_habbakuk, 
                target_size=input_size, 
                batch_size=batch_size,
                class_mode='categorical',subset='training')

Found 27 images belonging to 27 classes.


In [64]:
val_generator = gen_test.flow_from_directory(
                path_pretrain_habbakuk,  
                target_size=input_size,
                batch_size=batch_size,
                class_mode='categorical',subset='validation')

Found 0 images belonging to 27 classes.


In [65]:
weights_pretrain = model.fit_generator(
        train_generator,
        steps_per_epoch= train_generator.samples // batch_size,
        epochs=100,
        validation_data=val_generator,
        validation_steps=val_generator.samples // batch_size)
model.save_weights(f'data+.h5')

Epoch 1/100
2/2 [==============================] - 0s 29ms/step - loss: 0.6025 - accuracy: 0.8333
Epoch 2/100
2/2 [==============================] - 0s 29ms/step - loss: 0.4785 - accuracy: 0.8667
Epoch 3/100
2/2 [==============================] - 0s 10ms/step - loss: 0.6041 - accuracy: 0.8667
Epoch 4/100
2/2 [==============================] - 0s 26ms/step - loss: 0.8402 - accuracy: 0.7500
Epoch 5/100
2/2 [==============================] - 0s 26ms/step - loss: 0.6606 - accuracy: 0.7333
Epoch 6/100
2/2 [==============================] - 0s 10ms/step - loss: 0.7258 - accuracy: 0.7333
Epoch 7/100
2/2 [==============================] - 0s 24ms/step - loss: 0.7849 - accuracy: 0.7083
Epoch 8/100
2/2 [==============================] - ETA: 0s - loss: 0.2987 - accuracy: 1.00 - 0s 28ms/step - loss: 0.7501 - accuracy: 0.6000
Epoch 9/100
2/2 [==============================] - 0s 9ms/step - loss: 0.6466 - accuracy: 0.8000
Epoch 10/100
2/2 [==============================] - 0s 26ms/step - loss: 0.60